In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import ElectraModel,BertModel
import utils as keu # key_electra_utils.py 참고
from konlpy.tag import Hannanum
import pandas as pd
import ast
import re

model_new = SentenceTransformer(model_name_or_path='model/disc_book',tokenizer_path='model/tokenizer/')
model_old = SentenceTransformer(model_name_or_path='monologg/koelectra-base-v3-discriminator')

In [29]:
raw_data = pd.read_csv('data/raw/raw_book_info_list.csv',index_col=0)

raw_data.head(2)

,book_title,book_toc,book_intro,publisher
0,한 권으로 끝내는 메타버스 크리에이터,"['메타버스란', '왜 메타버스인가', '메타버스의 유형을 알아보자', '메타버스 ...",[],[]
1,Do it! 점프 투 파이썬: 라이브러리 예제 편,"['', '텍스트 다루기', '문자열을 줄여 표시하려면 textwrap shorte...",['이 책은 Do it 점프 투 파이썬 의 박응용 저자가 그동안 수많은 독자에게 받...,['실무에서 자주 쓰는 파이썬 라이브러리는 다 있다 필수 파이썬 라이브러리 개 엄선...


## [데이터 전처리] 단어 키워드 추출에 맞는 Input 데이터 생산

### 1. 도서 정보를 하나의 string으로 넣기

In [30]:
# book_info :str =keu.merge_series_to_str(raw_data.iloc[2323])
book_info :str =keu.merge_series_to_str(raw_data.iloc[1])

변환한 도서정보 :  Do it! 점프 투 파이썬: 라이브러리 예제 편


### 2. 일부 영단어를 한글로 변환하기

In [31]:
englist = pd.read_csv('preprocess/englist.csv')
book_info_trans = keu.trans_eng_to_han(book_info,englist=englist)

### 3. 문장 내 영단어 제거 및 영단어 모아두기

In [32]:
book_info_han = keu.find_han(' '.join(book_info_trans))
book_info_eng = keu.find_eng(' '.join(book_info_trans))

### Konlpy로 명사만 추출하기

In [33]:
from konlpy.tag import Hannanum

han = Hannanum()

# str or list에 따른 tokenizing 방법
if type(book_info_han) == list :
    han_nouns = han.nouns(' '.join(book_info_trans))
    
else :
    print('book_info type must be list.')

### Sentence Transformer로 유사도 검색

In [39]:
keu.key_extraction(book_info_han,model_old)

,0
공식,0.894798
병렬로,0.830597
파이썬으로,0.795139
비교하려면,0.790259
뽑으려면,0.786277
확인하려면,0.777321
바이너리,0.773035
테스트하려면,0.771322
읽으려면,0.770948
처리하려면,0.766809


In [40]:
keu.key_extraction(book_info_han,model_new,min_rank=20)

,0
파이썬으로,0.875315
병렬로,0.871060
읽으려면,0.863726
예제를,0.862628
것이다,0.853048
동시에,0.849964
사용하려면,0.839003
실행하려면,0.837551
저장하려면,0.837106
확인하려면,0.833603
